In [61]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres
from poliastro.twobody.thrust.change_a_inc import change_a_inc
from poliastro._math.linalg import norm
import poliastro.core.angles as angles
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# Define the two coplanar debris
a = 3000+6371 << u.km
ecc = 0 << u.one
inc = 10 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 10 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 6000+6371 << u.km
inc = 45 << u.deg
nu = 60 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

In [63]:
print(deb_1.epoch.iso)
print(deb_2.epoch.iso)
print(otv.epoch.iso)

2000-01-01 12:00:00.000
2000-01-01 12:00:00.000
2000-01-01 12:00:00.000


In [64]:
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [65]:
otv.argp

<Quantity 0. deg>

In [66]:
import CustomManeuvres
inc_change = CustomManeuvres.simple_inc_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

thrust_location mean:  179.999 deg
thrust_norm 3922.362414636926 m / s
y_thrust 1179.477125838908 m / s
z_thrust -3740.8235218169816 m / s
otv inc 44.99999999786559 deg
deb_2 inc 45.0 deg


In [69]:
otv.argp = 0 << u.deg

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [39]:
print(deb_1.nu << u.deg)
print(otv.nu << u.deg)
print(deb_2.nu << u.deg)
print((otv.nu + otv.argp) << u.deg)

# mean_anomaly = angles.nu_to_M(otv.nu, otv.ecc)

# print(mean_anomaly << u.deg)

179.9989999999999 deg
89.99918085912401 deg
172.07740403789592 deg
179.99975442439205 deg


In [59]:
# Propapage all a bit
t = 1000 * u.s
deb_1 = deb_1.propagate(t)
deb_2 = deb_2.propagate(t)
otv = otv.propagate(t)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [33]:
otv.argp << u.deg

<Quantity 90.00057357 deg>

In [60]:
import CustomManeuvres


inc_change = CustomManeuvres.simple_inc_change(otv, deb_1)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_1 inc', deb_1.inc << u.deg)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

thrust_location mean:  0.0 deg
zero deg transformation
thrust_norm -3922.362414074179 m / s
y_thrust -1179.477125600009 m / s
z_thrust -3740.82352130225 m / s
otv inc 15.162528189670601 deg
deb_1 inc 10.0 deg


In [42]:
inc_change.impulses

((<Quantity 3513.9909823 s>,
  <Quantity [ 1179.51457417,  2645.1533294 , -2645.15333027] m / s>),)

In [116]:
# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas

Target Delta:  31.72445180004123 deg
current_delta :  59.999999999999986 deg
dist:  28.275548199958756 deg
t_1:  2081.1542349395418 s
mean_anomaly_i:  82.98830235254066 deg
new delta:  31.724451800041223 deg


Number of impulses: 2, Total cost: 0.841563 km / s

In [18]:
hoh_phas.impulses

((<Quantity 33931.51750328 s>,
  <Quantity [297.99784075,  74.61079575,   0.        ] m / s>),
 (<Quantity 5255.47453664 s>,
  <Quantity [-283.91309118,  -71.08434613,    0.        ] m / s>))

In [117]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [18]:
# Wrong values, need to use mean anomaly to compare
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))
print('difference: ', (otv.nu - deb_2.nu).to(u.deg))
print('OTV.a: ', otv.a)
print('Deb2.a: ', deb_2.a)

OTV ANGLE:  -172.61536114641075 deg
DEB2 ANGLE:  157.19332482068768 deg
difference:  -329.8086859670984 deg
OTV.a:  9371.0 km
Deb2.a:  9371.0 km
